In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df = pd.read_pickle('/content/13_genre.pkl')
df

,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Description_length,Genre_first
Rank,,,,,,,,,,,,,
1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,127,Action
2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,127,Adventure
3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,157,Horror
4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,222,Animation
5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,179,Action
...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0,181,Crime
997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0,121,Horror
998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0,111,Drama


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
new_df = df[['Description','Genre_first']]
new_df.head()

,Description,Genre_first
Rank,,
1,A group of intergalactic criminals are forced ...,Action
2,"Following clues to the origin of mankind, a te...",Adventure
3,Three girls are kidnapped by a man with a diag...,Horror
4,"In a city of humanoid animals, a hustling thea...",Animation
5,A secret government agency recruits some of th...,Action


In [0]:
new_df.shape

(1000, 2)

### Vectorising the Description series for LDA to work on it because LDA needs a Document-term/word matrix as input

In [0]:
cv = CountVectorizer(max_df=0.9,min_df=2, stop_words='english')

In [0]:
dtm  = cv.fit_transform(new_df['Description'])
dtm

<1000x2232 sparse matrix of type '<class 'numpy.int64'>'
	with 11438 stored elements in Compressed Sparse Row format>

In [0]:
# Here the 2232 are the unique words in the vocab dictionary created 
dtm.shape

(1000, 2232)

In [0]:
cv.get_feature_names()[1000]

'imf'

In [0]:
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
# We are assuming that there are 5 topic(K) in the entire corpus . 
lda = LatentDirichletAllocation(n_components=5,random_state=101)

In [0]:
lda_fit  = lda.fit(dtm)

In [0]:
# Here the 10000 rows are reduced to 5 categories with same vocab length
lda_fit.components_.shape

(5, 2232)

In [0]:
# understanding each topics top 30 common words 
for id_value, value in enumerate(lda_fit.components_):
  print(f"The topic would be {id_value}")
  print([cv.get_feature_names()[index] for index in value.argsort()[-30:]])
  print("\n")
  print("\n")

The topic would be 0
['mother', 'human', 'years', 'war', 'save', 'agent', 'daughter', 'help', 'story', 'way', 'friends', 'takes', 'death', 'set', 'son', 'city', 'leads', 'mysterious', 'party', 'friend', 'stop', 'work', 'life', 'family', 'world', 'group', 'home', 'man', 'young', 'new']




The topic would be 1
['angeles', 'los', 'crime', 'discover', 'falls', 'child', 'team', 'son', 'police', 'murder', 'home', 'girl', 'drug', 'friends', 'true', 'american', 'mysterious', 'city', 'help', 'family', 'finds', 'world', 'new', 'love', 'war', 'man', 'life', 'father', 'woman', 'young']




The topic would be 2
['sets', 'ex', 'day', 'small', 'death', 'finds', 'star', 'town', 'relationship', 'time', 'mysterious', 'couple', 'home', 'friends', 'mother', 'story', 'year', 'old', 'gets', 'wife', 'school', 'family', 'boy', 'order', 'man', 'life', 'girl', 'young', 'new', 'world']




The topic would be 3
['bond', 'fall', 'soon', 'kill', 'starts', 'drug', 'mission', 'years', 'mysterious', 'college', 'force

In [0]:
results = lda_fit.transform(dtm)

In [0]:
# After transforming the data we have the same rows but with 5 topics that the LDA predicted this corpus has
results.shape

(1000, 5)

In [0]:
# findng max value among the array of rows 
# so result would look like : (random data)
# [0.2,0.04,0.005,0.1,0.09,0.7] --> argmax() : 4 (5th value)
# [0.289,0.6,0.057,0.1,0.06] --> argmax() : 1(2nd value)
new_df['LDA_pred_number']  = results.argmax(axis=1)
new_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Description,Genre_first,LDA_pred_number
Rank,,,
1,A group of intergalactic criminals are forced ...,Action,0
2,"Following clues to the origin of mankind, a te...",Adventure,0
3,Three girls are kidnapped by a man with a diag...,Horror,0
4,"In a city of humanoid animals, a hustling thea...",Animation,1
5,A secret government agency recruits some of th...,Action,3


In [0]:
df['LDA_pred_number'] = results.argmax(axis=1)
df.head()

,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Description_length,Genre_first,LDA_pred_number
Rank,,,,,,,,,,,,,,
1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,127,Action,0
2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,127,Adventure,0
3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,157,Horror,0
4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,222,Animation,1
5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,179,Action,3


# EDA in progess....

In [0]:
df[new_df['LDA_pred_number'] == 0]['Genre'].value_counts()[:5]

Action,Adventure,Sci-Fi     20
Crime,Drama,Thriller        10
Drama                        9
Comedy                       9
Action,Adventure,Fantasy     8
Name: Genre, dtype: int64

In [0]:
df[new_df['LDA_pred_number'] == 1]['Genre'].value_counts()[:5]

Crime,Drama,Mystery        12
Drama                      11
Action,Adventure,Sci-Fi     9
Comedy,Romance              9
Comedy,Drama,Romance        8
Name: Genre, dtype: int64

In [0]:
df[new_df['LDA_pred_number'] == 2]['Genre'].value_counts()[:5]

Drama                      14
Comedy,Drama               10
Action,Adventure,Sci-Fi    10
Drama,Romance               8
Comedy                      8
Name: Genre, dtype: int64

In [0]:
df[new_df['LDA_pred_number'] == 3]['Genre'].value_counts()[:5]

Comedy,Drama,Romance    10
Comedy,Romance           8
Drama,Romance            8
Comedy                   8
Drama                    7
Name: Genre, dtype: int64

In [0]:
df[new_df['LDA_pred_number'] == 4]['Genre'].value_counts()[:5]

Drama,Romance                 8
Drama                         7
Animation,Adventure,Comedy    7
Action,Adventure,Sci-Fi       6
Biography,Drama,History       6
Name: Genre, dtype: int64

In [0]:
def converting_LDA_genre(data):
  if data == 0:
    return "Action,Adventure,Sci-Fi"
  if data == 1:
    return "Crime,Drama,Mystery"
  if data ==2:
    return "Drama"
  if data ==3:
    return "Comedy,Drama,Romance"
  else:
    return "Drama,Romance"

In [0]:
df['LDA_pred_number'] = df['LDA_pred_number'].apply(converting_LDA_genre)
df.head()

,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Description_length,Genre_first,LDA_pred_number
Rank,,,,,,,,,,,,,,
1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,127,Action,"Action,Adventure,Sci-Fi"
2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,127,Adventure,"Action,Adventure,Sci-Fi"
3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,157,Horror,"Action,Adventure,Sci-Fi"
4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,222,Animation,"Crime,Drama,Mystery"
5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,179,Action,"Comedy,Drama,Romance"


In [0]:
df.to_pickle("LDA_5_topic.pkl")

In [0]:
df[df['LDA_pred_number'] == "Drama,Romance"]

,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Description_length,Genre_first,LDA_pred_number
Rank,,,,,,,,,,,,,,
10,Passengers,"Adventure,Drama,Romance",A spacecraft traveling to a distant colony pla...,Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,...",2016,116,7.0,192177,100.01,41.0,184,Adventure,"Drama,Romance"
18,Jason Bourne,"Action,Thriller",The CIA's most dangerous former operative is d...,Paul Greengrass,"Matt Damon, Tommy Lee Jones, Alicia Vikander,V...",2016,123,6.7,150823,162.16,58.0,113,Action,"Drama,Romance"
24,Trolls,"Animation,Adventure,Comedy","After the Bergens invade Troll Village, Poppy,...",Walt Dohrn,"Anna Kendrick, Justin Timberlake,Zooey Deschan...",2016,92,6.5,38552,153.69,56.0,148,Animation,"Drama,Romance"
29,Bad Moms,Comedy,When three overworked and under-appreciated mo...,Jon Lucas,"Mila Kunis, Kathryn Hahn, Kristen Bell,Christi...",2016,100,6.2,66540,113.08,60.0,197,Comedy,"Drama,Romance"
30,Assassin's Creed,"Action,Adventure,Drama",When Callum Lynch explores the memories of his...,Justin Kurzel,"Michael Fassbender, Marion Cotillard, Jeremy I...",2016,115,5.9,112813,54.65,36.0,171,Action,"Drama,Romance"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,Dark Places,"Drama,Mystery,Thriller",Libby Day was only eight years old when her fa...,Gilles Paquet-Brenner,"Charlize Theron, Nicholas Hoult, Christina Hen...",2015,113,6.2,31634,NaN,39.0,244,Drama,"Drama,Romance"
982,Annie,"Comedy,Drama,Family","A foster kid, who lives with her mean foster m...",Will Gluck,"Quvenzhané Wallis, Cameron Diaz, Jamie Foxx, R...",2014,118,5.3,27312,85.91,33.0,188,Comedy,"Drama,Romance"
991,Underworld: Rise of the Lycans,"Action,Adventure,Fantasy",An origins story centered on the centuries-old...,Patrick Tatopoulos,"Rhona Mitra, Michael Sheen, Bill Nighy, Steven...",2009,92,6.6,129708,45.80,44.0,131,Action,"Drama,Romance"
